<div>
    <h2><center>Global Glacier Velocity point Tracker</center></h2>
</div>

In [ ]:
# Fetch the path for essential python scripts

import os
import sys
path =  os.getcwd()
# Avoid Windows users to have issues with how paths are written
path = path.replace('\\','/')

# Import python scripts from notebooks folder
sys.path.append(path + '/scripts')
%matplotlib widget

In [ ]:
# Import all the necessary packages
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.path as mpath
import markdown 

matplotlib.rcParams['figure.figsize'] = [9, 5]
matplotlib.rcParams["figure.autolayout"] = True

from velocity_widget import ITSLIVE
velocity_widget = ITSLIVE()
plt.close()
from ipywidgets import widgets, HTML, Output
import xarray as xr
import numpy as np
import time
from pyproj import Transformer
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import geopandas as gpd
import ipyleaflet as ipl
import math
import pandas as pd
import glob
import seaborn
import warnings
import rasterio as rio


### Instructions

<br>

This notebook will allow you to track pixels at the surface of your AOI, depending on their displacement from the velocity dataset ITS-LIVE.

<br>

1. Choose the type of dataset you want to work with. Just make sure you already downloaded some data.

2. Select a date range. Make sure this range is at least the one you selected when you downloaded your data. It can be smaller if you want.



In [ ]:
# Select the type of dataset you want to download: 'Yearly' or 'Subyearly'
type_dataset = "Yearly"

# Select your date range (careful the datacubes get heavy quickly)
sdate = '1998-01-01'
edate = '2018-01-01'



#### Select approximately the area you want to work on (if you already have it downloaded)

<br>

This section should select automatically the data closest to your AOI.
Run the following cell.
Instructions: same as for data downloading for Yearly dataset:
- Left click once to put a marker
- Left click a second time to plot the AOI's boundaries.

In [ ]:

dates_range = widgets.SelectionRangeSlider(
    options=[i for i in range(546)],
    index=(1, 120),
    continuous_update=False,
    description='Interval (days): ',
    orientation='horizontal',
    layout={'width': '90%',
            'display': 'flex'},
    style={'description_width': 'initial'})

variables =  widgets.Dropdown(
    options=['v', 'v_error', 'vx', 'vy'],
    description='Variable: ',
    disabled=False,
    value='v',
    layout={'width': '20%',
            'display': 'flex'},
    style={'description_width': 'initial'})

plot_type =  widgets.Dropdown(
    options=['location', 'satellite'],
    description='Plot By: ',
    disabled=False,
    value='location',
    layout={'width': '20%',
            'display': 'flex'},
    style={'description_width': 'initial'})

plot_button =  widgets.Button(
    description='Plot',
    button_style='primary',
    icon='line-chart',
    style={'description_width': 'initial'})

get_points =  widgets.Button(
    description='Get points',
    button_style='primary',
    icon='line-chart',
    style={'description_width': 'initial'})

clear_button =  widgets.Button(
    description='Clear Points',
    # button_style='warning',
    icon="trash",
    style={'description_width': 'initial'})

latitude = widgets.BoundedFloatText(
    value=0.0,
    min=-90.0,
    max=90.0,
    step=0.1,
    description='Lat: ',
    disabled=False,
    style={'description_width': 'initial'},
    layout={'width': '20%',
            'display': 'flex'},
)

longitude = widgets.BoundedFloatText(
    value=0.0,
    min=-180.0,
    max=180.0,
    step=0.1,
    description='Lon: ',
    disabled=False,
    style={'description_width': 'initial'},
    layout={'width': '20%',
            'display': 'flex'},
)

add_button =  widgets.Button(
    description='Add Point',
    # button_style='info',
    icon="map-marker",
    style={'description_width': 'initial'})

include_running_mean =  widgets.Checkbox(
            value=False,
            description="Include Running Mean",
            style={'description_width': 'initial'},
            disabled=False,
            indent=False,
            tooltip="Plot running mean through each time series",
            layout=widgets.Layout(width="25%"),
        )

export_button = widgets.Button(
    description='Export Data',
    # button_style='info',
    icon="file-export",
    style={'description_width': 'initial'})

data_link = widgets.HTML(
    value="<br>"
)

# If this congiguration changes we need to rerun the cell.
config = { 
    "plot": "v", # or other ITS_LIVE variables: vx, vy ...
    "min_separation_days": 1,
    "max_separation_days": 90,
    "color_by": "location", # valid values: satellite, points
    "verbose": True, # print operations
    "runnig_mean": True,
    "coords": {
        "latitude": latitude,
        "longitude": longitude
    },
    "data_link": data_link
}


def downloader(whatever):
    print('Downloading...')
    global pathsave
    ######### YEARLY DATASET DOWNLOAD ########
    if type_dataset == 'Yearly':

            # Create list of years for the date range chosen earlier
            list_years = np.arange(int(sdate.split('-')[0]), int(edate.split('-')[0])+1)

            # Create path to the files
            pathsave = velocity_widget.path_yearly_datacubes
            os.makedirs(pathsave, exist_ok = True)
            


            for Y in range(len(list_years)):

                    # Generate URL for the nc file
                    url = f'{velocity_widget.url_region[0]}{int(list_years[Y])}.nc#mode=bytes'

                    # Load datacube according to prerequisites (time, space and variables)
                    start = time.time()
                    xrds = xr.open_dataset(url
                                            ).sel(x=slice(velocity_widget.xmin_proj, velocity_widget.xmax_proj),
                                                y=slice(velocity_widget.ymax_proj, velocity_widget.ymin_proj)).load()

                    print(f"downloaded {list_years[Y]} spatial slice {time.time()-start:8.1f} seconds")
                    xrds.to_netcdf(f"{pathsave}{list_years[Y]}.nc")




    ######## SUBYEARLY DATASET DOWNLOAD ########
    else:

            # Create path to the files
            pathsave = velocity_widget.path_subyearly_datacubes
            os.makedirs(pathsave, exist_ok = True)

            # Get the cube address
            cubes = velocity_widget.dct.addresses
            cubes = [*set(cubes)]

            # List of variables to drop for the download (we drop everything but the variables written below)
            variables_drop = [ele for ele in list(
                    xr.open_dataset(cubes[0], engine='zarr').variables
                    ) if ele not in ['mid_date','x','y','acquisition_date_img1', 'acquisition_date_img2', 'date_center', 'date_dt', 'satellite_img1','satellite_img2', 'v','vx','vy','roi_valid_percentage']
            ]

            
            for n in range(len(cubes)):

                    # Get the cube's URL
                    url = cubes[n]

                    # Load indices of slices above the quality threshold
                    valid = xr.open_dataset(cubes[n], engine='zarr').roi_valid_percentage.values

                    # Grab the time values
                    t = xr.open_dataset(cubes[n], engine='zarr').mid_date.values

                    # Create a time mask, based on the validity of layers and the custom date-range
                    t_mask = np.logical_and(valid>threshold, np.logical_and(t>np.datetime64(sdate), t<np.datetime64(edate)))
                    
                    # Load datacube according to prerequisites (time, space and variables)
                    start = time.time()
                    xrds = xr.open_dataset(url,
                                            engine='zarr',
                                            drop_variables=variables_drop
                                            ).sel(mid_date=t_mask,
                                                x=slice(velocity_widget.xmin_proj, velocity_widget.xmax_proj),
                                                y=slice(velocity_widget.ymax_proj, velocity_widget.ymin_proj)).load()

                    print(f"downloaded {cubes[n].split('/')[-1].split('.')[0]} spatial slice {time.time()-start:8.1f} seconds")
                    xrds.to_netcdf(f"{pathsave}{cubes[n].split('/')[-1].split('.')[0]}_{sdate}_{edate}.nc")
    print('Done ! You can hit "plot" now')

def plotter(whatever):
    list_files = glob.glob(f'{pathsave}*.nc')

    fig, ax = plt.subplots(len(list_files), figsize=(10,10))

    if len(list_files) == 1:
        if type_dataset == "Yearly":
            ax.pcolormesh(xr.open_dataset(list_files[0]).x.values,xr.open_dataset(list_files[0]).y.values,xr.open_dataset(list_files[0]).v.values)
        else:
            ax.pcolormesh(xr.open_dataset(list_files[0]).x.values,xr.open_dataset(list_files[0]).y.values,np.nanmean(xr.open_dataset(list_files[0]).v.values,axis = 0))
    else:
        if type_dataset == "Yearly":
            for i in range(len(list_files)):
                ax[i].pcolormesh(xr.open_dataset(list_files[i]).x.values,xr.open_dataset(list_files[i]).y.values,xr.open_dataset(list_files[i]).v.values)
        else:
            for i in range(len(list_files)):
                ax[i].pcolormesh(xr.open_dataset(list_files[i]).x.values,xr.open_dataset(list_files[i]).y.values,np.nanmean(xr.open_dataset(list_files[i]).v.values,axis = 0))


def update_variable(change):
        if change['type'] == 'change' and change['name'] == 'value':
            config["plot"] = variables.value
            velocity_widget.set_config(config)
            velocity_widget.plot_time_series()
            
def update_range(change):
        if change['type'] == 'change' and change['name'] == 'value':
            start, end = change['new']
            config["min_separation_days"] = start
            config["max_separation_days"] = end
            velocity_widget.set_config(config)
            velocity_widget.plot_time_series()
            
def update_plottype(change):
        if change['type'] == 'change' and change['name'] == 'value':
            config["color_by"] = plot_type.value
            velocity_widget.set_config(config)
            velocity_widget.plot_time_series()
            
def update_mean(change):
        if change['type'] == 'change' and change['name'] == 'value':
            config["running_mean"] = include_running_mean.value
            velocity_widget.set_config(config)
            velocity_widget.plot_time_series()
            
def add_point(event):
    coords = (latitude.value, longitude.value)
    velocity_widget.add_point(coords)
    
def export_ts(event):
    velocity_widget.export_data()

get_points.on_click(velocity_widget.plot_time_series)
plot_button.on_click(plotter)
clear_button.on_click(velocity_widget.clear_points)

export_button.on_click(downloader)


add_button.on_click(add_point)
dates_range.observe(update_range, 'value')
plot_type.observe(update_plottype, 'value')
variables.observe(update_variable, 'value')
include_running_mean.observe(update_mean, 'value')

layout = widgets.Layout(align_items='stretch',
                        display='flex',
                        flex_flow='row wrap',
                        border='none',
                        grid_template_columns="repeat(auto-fit, minmax(720px, 1fr))",
                        # grid_template_columns='48% 48%',
                        width='99%',
                        height='100%')

velocity_widget.set_config(config)

velocity_widget.fig.canvas.capture_scroll = True

# We render the widget
widgets.GridBox([
                widgets.VBox([velocity_widget.map,
                            widgets.HBox([latitude, longitude, add_button, clear_button], layout=widgets.Layout(align_items="flex-start",
                                                                                                                flex_flow='row wrap'))],
                            layout=widgets.Layout(min_width="100%",
                                                    display="flex",
                                                    # height="100%",
                                                    # max_height="100%",
                                                    max_width="100%"))],

                layout=layout)

**Function to get the file's names in the folder corresponding to your AOI**

In [ ]:
def folder_finder(name_region):
        # Gather list of folders in the region of the AOI
        list_folders = glob.glob(f'{path}/Datacubes/{type_dataset}/{name_region[0]}/*')
        # Make sure that Windows users won't encounter troubles with the '\\' character in the path
        list_folders = [(list_folders[i].replace('\\','/')) for i in range(len(list_folders))]
        # Calculate the center-point of the folders based on their respective AOI
        center_folders = [((float(list_folders[i].split('/')[-1].split('_')[2])-float(list_folders[i].split('/')[-1].split('_')[0]))/2+float(list_folders[i].split('/')[-1].split('_')[0]),
                        (float(list_folders[i].split('/')[-1].split('_')[3])-float(list_folders[i].split('/')[-1].split('_')[1]))/2+float(list_folders[i].split('/')[-1].split('_')[1])) for i in range(len(list_folders))]
        # Calculate the distance between the center of your AOI and the folders' AOI's centers
        dist = [np.sqrt((center_folders[i][0]-velocity_widget.point_center.coords[0][0])**2 + (
                center_folders[i][1]-velocity_widget.point_center.coords[0][1])**2) for i in range(len(center_folders))]
        # Get the index of the folder the closest to your AOI
        list_files = glob.glob(f'{list_folders[np.where(dist==min(dist))[0][0]]}/*.nc')
        list_files = [(list_files[i].replace('\\','/')) for i in range(len(list_files))]
        # Update the path to save files
        if type_dataset == 'Yearly':
                pathsave = velocity_widget.path_yearly_datacubes.replace(velocity_widget.path_yearly_datacubes.split('/')[-2], list_files[0].split('/')[-2] ).replace('Datacubes','Output')

        else:
                pathsave = velocity_widget.path_subyearly_datacubes.replace(velocity_widget.path_subyearly_datacubes.split('/')[-2], list_files[0].split('/')[-2] ).replace('Datacubes','Output')

        # Create output folder
        os.makedirs(pathsave, exist_ok = True)

        return list_files, pathsave
    

In [ ]:
# Get the list of files in the folder corresponding to your AOI
list_files, pathsave = folder_finder(velocity_widget.name_region)

# Get the coordinates of your AOI's boundaries
coords = [(float(list_files[0].split('/')[-2].split('_')[1]), (float(list_files[0].split('/')[-2].split('_')[0]))), (
           float(list_files[0].split('/')[-2].split('_')[3]), (float(list_files[0].split('/')[-2].split('_')[2]))) ]

## Define your analysis Frequency:

<br>

In the next cell, you will choose on which time interval to calculate variable's means.

The variable "TYPE" allows you to choose several options:
- "Y" for year
- "M" for end-of-month
- "MS" for start-of-month
- 'D" for day

<br>

The variable "amount" allows you to choose over how many of your "TYPE" variable you want to average.

For example, a mean every 5 months starting at the beginning of each month would have the variables:
- TYPE = "MS"
- amount = 5

<br>

Finally, "delta" calculates the time difference between each step.

In [ ]:
# Select the variable type
TYPE = 'Y'

# Select the amount of units you want to average for (eg: if TYPE = "Y" and amount = 1, average every 1 year)
# I personnally like to have a 5 days variable for subyearly (TYPE = 'D', amount = 5)
amount = 1

# Get timestep
delta = np.timedelta64(amount, TYPE)

**Functions to import the datasets**

In [ ]:
def importer(list_files, datatype, TYPE, sdate, edate):
    
    if datatype == 'Yearly':   
        
        # Initialize an array that will host the dates 
        years = np.zeros((len(list_files)), dtype='datetime64[s]')
        list_years = np.zeros((len(list_files)), dtype='int')
        
        # Get the date of each file if they are in the boundaries in the starting date and ending date, and update the list of files to process
        for i in range(0,len(list_files)):
            years = [pd.to_datetime(list_files[i].split('/')[-1].split('.')[0],format='%Y') for i in range(len(list_files))]
            list_years = [list_files[i].split('/')[-1].split('.')[0] for i in range(len(list_files)) if np.logical_and(np.datetime64((list_files[i].split('/')[-1].split('.')[0]))>=np.datetime64(sdate), np.datetime64((list_files[i].split('/')[-1].split('.')[0]))<=np.datetime64(edate))]
            list_files = [list_files[i] for i in range(len(list_files)) if np.logical_and(np.datetime64((list_files[i].split('/')[-1].split('.')[0]))>=np.datetime64(sdate), np.datetime64((list_files[i].split('/')[-1].split('.')[0]))<=np.datetime64(edate))]
        
        # Initialize a dataset on which we will append the others
        # List of variables to drop for the download (we drop everything but the variables written below)
        drop = [ele for ele in list(
                xr.open_dataset(list_files[0]).variables
                ) if ele not in ['x','y','vx','vy','v']
        ]

                    
        ds = xr.open_dataset(list_files[0], 
                             drop_variables=drop
                             )
        
        # Loop through every file, which we will append to the existing dataset
        for i in range(1,len(list_files)):

            print(f'Opening {list_files[i]} ')
            
            drop = [ele for ele in list(
                    xr.open_dataset(list_files[i]).variables
                    ) if ele not in ['x','y','vx','vy','v']
            ]
              
            dtemp = xr.open_dataset(list_files[i],
                                    drop_variables=drop)  
                                            
            ds = xr.concat((ds,dtemp), 'time')
        
        # Assign the dates to the time dimension
        ds = ds.assign_coords({'time':years})
        
    else:
        
        list_years = []
        years = []
        
        # Grab the time values
        t = xr.open_dataset(list_files[0]).mid_date.values

        # Create a time mask, based on the validity of layers and the custom date-range
        t_mask = np.logical_and(t>=np.datetime64(sdate), t<=np.datetime64(edate))

        ds = xr.open_dataset(list_files[0]).sel(mid_date=t_mask)
        ds = ds.sortby(ds.mid_date).resample(mid_date=f'{amount}{TYPE}').mean(dim="mid_date",skipna = True)

        for i in range(len(list_files)):
            t = xr.open_dataset(list_files[i]).mid_date.values
            t_mask = np.logical_and(t>=np.datetime64(sdate), t<=np.datetime64(edate))
            temp = xr.open_dataset(list_files[i]).sel(mid_date=t_mask)
            temp = temp.sortby(temp.mid_date).resample(mid_date=f'{amount}{TYPE}').mean(dim="mid_date", skipna = True)
            ds = xr.concat((ds,temp),dim='mid_date')
        
        # Sort the time index to allow time slicing
        ds = ds.sortby(ds.mid_date).resample(mid_date=f'{amount}{TYPE}').mean(dim="mid_date", skipna = True)
        
        # Rename the time dimension so we don't have to modify the entire code
        ds = ds.rename({'mid_date': 'time'})
    
    return ds, years, list_years

In [ ]:
# Import the files as an xarray object, their dates and years
ds, years, list_years = importer(list_files, type_dataset, TYPE, sdate, edate)

#### Select the points you want to track

<br>

To do that:
- Left click to select points to track
- Right click to change the group of points

By changing the group of points, you can track several transects at once.
For example you can select several transects along flow to assess how the ice flows along the glacier.

**Cell to plot the map and get coordinates on click and store them in an array**


In [ ]:

def handle_click(**kwargs):
    global i
    global n
    global c
    # If left click, add marker, grab coordinates and store in the coordinates array
    # Ideally, I would like 'click' to be 'leftclick' if that exists
    if kwargs.get('type') == 'click':
        
        icon = ipl.AwesomeIcon(name='fa-cog', marker_color = colors[c])
        m.add_layer(ipl.Marker(location=kwargs.get('coordinates'), icon=icon))
        coordinates[n,i,:] = (kwargs['coordinates'])
        i += 1

    
    # If right click, change color of marker, change column in coordinates, reinitialize the row counter
    elif kwargs.get('type') == 'contextmenu':
        i = 0
        n += 1
        c += 1
        if c > len(colors)-1:
            c = 0




# Initialize coordinates array
coordinates=np.full((1000,1000,2), np.nan)

# Initialize indices
i = 0
n = 0
c = 0
# Create a rotating array of colors to differenciate the markers
colors = ['red', 'green', 'blue', 'purple', 'gray', 'orange', 'beige']

# plot the map
m = ipl.Map(
    basemap=ipl.basemap_to_tiles(ipl.basemaps.Esri.WorldImagery),
    center=((coords[1][0]-coords[0][0])/2 + coords[0][0],
            (coords[1][1]-coords[0][1])/2 + coords[0][1]),
    zoom=7,
    scroll_wheel_zoom=True
    )

# plot the AOI
rectangle = ipl.Rectangle(bounds=((coords[1][0], coords[1][1]), (coords[0][0], coords[0][1])))
m.add_layer(rectangle)

m.on_interaction(handle_click)
m

#### Calculate the displacement of each point selected over time

In [ ]:
    
def get_vels(pts, ind_nan):
    
    # Create time position array
    pts_time = np.zeros((ds.time.shape[0], ind_nan, pts.shape[1]), dtype = int)
    pts_time[0] = pts

    
    for i in range(1,ds.time.shape[0]):
        
        time_step = ds.time.values[i] - ds.time.values[i-1]
        time_step = time_step.astype('timedelta64[D]')
        time_step = time_step / np.timedelta64(365, 'D')
        
        
        for j in range(0, ind_nan):
            
            if np.isnan(ds.vx.isel(time = i).sel(x=pts_time[i-1,j,0], y=pts_time[i-1,j,1], method="nearest").values):
                pts_time[i,j,0] = pts_time[i-1,j,0]
            else:
                pts_time[i,j,0] = pts_time[i-1,j,0] + ds.vx.isel(time = i).sel(x=pts_time[i-1,j,0], y=pts_time[i-1,j,1], method="nearest").values*time_step
                
            if np.isnan(ds.vy.isel(time = i).sel(x=pts_time[i-1,j,0], y=pts_time[i-1,j,1], method="nearest").values):
                pts_time[i,j,1] = pts_time[i-1,j,1]
            else:
                pts_time[i,j,1] = pts_time[i-1,j,1] + ds.vy.isel(time = i).sel(x=pts_time[i-1,j,0], y=pts_time[i-1,j,1], method="nearest").values*time_step
          
    return pts_time


# Convert the coordinates to the dataset's projection
transformer = Transformer.from_crs(4326, int(velocity_widget.proj_region[0]))
coords_proj = np.array([[pt for pt in transformer.itransform(coordinates[i])] for i in range(coordinates.shape[0])])

# Determine how many points there are per transect
inds_nan = [np.where(np.isnan(coords_proj[i,:,0]))[0][0] for i in range(0,n+1)]

# Fill the array with points taken from the map
pts_total = np.full((coords_proj.shape[0], len(ds.time), coords_proj.shape[1], 2), np.nan)

for i in range(0,n+1):
    temp = get_vels(coords_proj[i,0:inds_nan[i]], inds_nan[i])
    pts_total[i,:,0:inds_nan[i],:] = temp   

#### Plot the displacements
- First position of points: orange
- Last position of points: black
- Intermediate positions: gradient from blue to white

In [ ]:
import matplotlib.pyplot as plt

# Used for the colors of the scatter plots
time = np.arange(0,len(ds.time))
# Background image
prog = np.nanmean(ds[list(ds.keys())[0]].values, axis = 0)

# plot the displacement of points
plt.figure(num = 2, figsize=(10,10))
plt.pcolormesh(ds.x, ds.y, prog, cmap = 'icefire')
b = plt.colorbar()
b.set_label('Velocity [m/yr]')

# Plot each timestep
for i in range(0,n+1):
    for j in range(0, inds_nan[i]):
        plt.scatter(pts_total[i,:,j,0],pts_total[i,:,j,1], c=time, cmap = 'Blues_r')
    
    # Plot the first and last positions: first is orange, last is black
    plt.plot(pts_total[i,0,:,0], pts_total[i,0,:,1],'-o', color='orange')
    plt.plot(pts_total[i,-1,:,0], pts_total[i,-1,:,1],'-o', color='black')

os.makedirs(f'{pathsave}Figures/Tracker', exist_ok = True)
plt.savefig(f'{pathsave}Figures/Tracker')


# Strain Rates

<br>

This section will be ran as it is. It uses the same time-range as for the pixel tracker.
However you have some options ! 

<br>

**Instructions:**

Run the next two cells. If you have an ice-thickness tif for you AOI, change "flux_div" to "True" (in the next cell) and indicate the path to:
- the path to ice thickness tif file
- the path to boundary file (supposed to be a 2 columns, csv file. See the template in the "Example" folder)

<br>

**WARNING:** Make sure the boundary and ice thickness are IN THE SAME PROJECTION as your downloaded datacubes.



In [ ]:
flux_div = False

path_thickness = f'{path}/Example/Thickness.tif'
path_boundaries = f'{path}/Example/Boundaries.csv'

In [ ]:
if flux_div:

        # Import thickness boundaries
        boundary_points = np.genfromtxt(path_boundaries, delimiter=',')
        boundary_points = boundary_points[1:]
        boundary_points = boundary_points[:,2:]

        # Create a mask for thickness
        xx = ds.x
        yy = ds.y
        xxx = np.linspace(0, len(xx), len(xx)+1)
        yyy = np.linspace(0, len(yy), len(yy)+1)
        gridx, gridy = np.meshgrid(xx, yy)

        X = np.vstack((gridx.ravel(),gridy.ravel())).T

        p = mpath.Path(np.vstack((boundary_points[:,0], boundary_points[:,1])).T)
        glacier_mask = p.contains_points(X)
        glacier_mask = np.hstack([glacier_mask]).reshape(ds.vx.shape[1],ds.vx.shape[2])


        # Import thickness tif which we will use to define our search window for the xarray
        with rio.open(path_thickness) as src:
                thickness = src.read()[0]
                height_thickness = thickness.shape[0]
                width_thickness = thickness.shape[1]
                cols_thickness, rows_thickness = np.meshgrid(np.arange(width_thickness), np.arange(height_thickness))
                xs_thickness, ys_thickness = rio.transform.xy(src.transform, rows_thickness, cols_thickness)
                lons_thickness= np.array(xs_thickness)[0]
                lats_thickness = np.array(ys_thickness)[:,0]

In [ ]:
# Grabs the closest indices of an array based on the input
def closest_value(pty, ptx, array):
    
    minx = np.abs(lons_thickness-ptx)
    miny = np.abs(lats_thickness-pty)
    
    col = np.where(minx == minx.min())[0][0]
    row = np.where(miny == miny.min())[0][0]
    
    return row, col



# Calculate strains and dynamic thinning
def Strains_Calculator(datacube, sdate, edate, upscale, freq, Plotting, flux_div):

    xx = ds.x
    yy = ds.y
    
    # We suppress the warning for means of empty slice otherwise it pops-up often
    with warnings.catch_warnings():
        warnings.filterwarnings(action='ignore', message='Mean of empty slice')
        vxtot = np.nanmean(datacube.vx.sel(time = slice(sdate, edate)), axis = 0)
        vytot = np.nanmean(datacube.vy.sel(time = slice(sdate, edate)), axis = 0)

    # Calculate spatial steps
    dx = np.abs(xx[0] - xx[upscale])
    dy = np.abs(yy[0] - yy[upscale]) 
    
    # Calculate spatial steps
    dx = np.abs(xx[0] - xx[upscale])
    dy = np.abs(yy[0] - yy[upscale])
    
    # Rescale X and Y
    YY = yy.values[::upscale]
    YY = YY[1:-1]
    XX = xx.values[::upscale]
    XX = XX[1:-1]
    gridX, gridY = np.meshgrid(XX,YY)

    #### Strain Rates #### 
    
    # Prepare arrays for the strain calculation
    vx = vxtot[::upscale, ::upscale]
    vy = vytot[::upscale, ::upscale]
      
    # Calculate the spatial step
    dx = np.abs(xx[0] - xx[upscale])
    dy = np.abs(yy[0] - yy[upscale])
    xxx = np.linspace(0, len(xx), len(xx)+1)
    yyy = np.linspace(0, len(yy), len(yy)+1)   
    
    # Rescale rows and cols
    dyy = yyy[::upscale]
    dyy = dyy[1:-1]
    dxx = xxx[::upscale]
    dxx = dxx[1:-1]
    
    # Calculate velocity derivatives components
    duy = (vx[2:,1:-1] - vx[:-2,1:-1])/(2*dx.values)
    dux = (vx[1:-1,2:] - vx[1:-1,:-2])/(2*dy.values)
    dvy = (vy[2:,1:-1] - vy[:-2,1:-1])/(2*dx.values)
    dvx = (vy[1:-1,2:] - vy[1:-1,:-2])/(2*dy.values) 

    # Calculate the strain components
    exx = dux
    eyy = dvy
    # Based on the assumption of flux divergence = 0
    ezz = -exx-eyy
    exy = 0.5*(duy + dvx)
    
    # Calculate the eigenvalues of the deviatoric stress tensor
    e1 = 0.5*(exx+eyy) + 0.5*np.sqrt(4*exy**2 + (exx-eyy)**2)
    e2 = 0.5*(exx+eyy) - 0.5*np.sqrt(4*exy**2 + (exx-eyy)**2)

    if flux_div:
    #### Flux Divergence ####
        
        # Scale the thickness to the dataset
        thickness_scaled = np.zeros(vxtot.shape)
        
        # Filter out non-glacier cells
        thickness_scaled[glacier_mask==False] = np.nan
        
        # Resize the thickness to the dataset
        for j in range(vxtot.shape[0]):
            for i in range(vxtot.shape[1]):
                if ~np.isnan(thickness_scaled[j,i]):    
                    row, col = closest_value(yy.values[j], xx.values[i], thickness)
                    thickness_scaled[j,i] = thickness[row,col]   
        
        # Filter out erratic values
        thickness_scaled[thickness_scaled==np.nanmin(thickness_scaled)]=np.nan
        
        # Calculate thickness components qx and qy
        qx = vxtot[::upscale,::upscale]*thickness_scaled[::upscale,::upscale]
        qy = vytot[::upscale,::upscale]*thickness_scaled[::upscale,::upscale]
    
        # Calculate dqx and dqy
        dqx = (qx[2:,1:-1] - qx[:-2,1:-1])/(2*dx.values)
        dqy = (qy[1:-1,2:] - qy[1:-1,:-2])/(2*dy.values)
        
        # Calculate flux divergence
        divflux = dqx + dqy
    
    else:
        # Ensure divflux at least exists
        divflux = np.zeros(ezz.shape)
        divflux[divflux==0] = np.nan
    
    
    if Plotting:       

        # Make a colorbar for the plots
        co = seaborn.diverging_palette(250, 30, l=65, center="dark", as_cmap=True)

        # plot flux divergence
        fig = plt.figure(num=1, clear=True, figsize=[10,10])
        ax = fig.add_subplot()
        f = ax.pcolormesh(gridX, gridY, divflux, cmap = co, vmin = -100, vmax = 100)
        fig.colorbar(f, ax = ax)
        ax.set_title('Flux Divergence [m] '+sdate + ' / ' +edate, fontsize=18)
        fig.savefig(f'{pathsave}Figures/Flux_div/{amount}_{TYPE}/{upscale}_{sdate}_{edate}.png', dpi = 400)
                    
 
        # plot 1st eigenvalue
        fig = plt.figure(num=1, clear=True, figsize=[10,10])
        ax = fig.add_subplot()
        f = ax.pcolormesh(gridX, gridY, e1, cmap = co, vmin = -1, vmax = 1)
        fig.colorbar(f, ax = ax)
        ax.set_title('e1 '+sdate + ' / ' +edate, fontsize=18)
        fig.savefig(f'{pathsave}Figures/e1/{amount}_{TYPE}/{upscale}_{sdate}_{edate}.png', dpi = 400)

        # plot 2nd eigenvalue
        fig = plt.figure(num=1, clear=True, figsize=[10,10])
        ax = fig.add_subplot()
        f = ax.pcolormesh(gridX, gridY, e2, cmap = co, vmin = -1, vmax = 1)
        fig.colorbar(f, ax = ax)
        ax.set_title('e2 '+sdate + ' / ' +edate, fontsize=18)
        fig.savefig(f'{pathsave}Figures/e2/{amount}_{TYPE}/{upscale}_{sdate}_{edate}.png', dpi = 400)

        # plot ezz eigenvalue
        fig = plt.figure(num=1, clear=True, figsize=[10,10])
        ax = fig.add_subplot()
        f = ax.pcolormesh(gridX, gridY, ezz, cmap = 'bwr', vmin = -0.1, vmax = 0.1)
        fig.colorbar(f, ax = ax)
        ax.set_title('\u03B5zz '+sdate + ' / ' +edate, fontsize=18)
        fig.savefig(f'{pathsave}Figures/ezz/{amount}_{TYPE}/{upscale}_{sdate}_{edate}.png', dpi = 400)


        
    return ezz, divflux, e1, e2


def Runner(Plotting, upscale, sdate, edate, flux_div):

    # Create host subfolders
    list_subfolders = ['Strains', 'Figures/e1', 'Figures/e2', 'Figures/ezz', 'Figures/Flux_div']
    [os.makedirs(f'{pathsave}{list_subfolders[i]}/{amount}_{TYPE}/', exist_ok = True) for i in range(len(list_subfolders))]
    
    # Create a template array for the strain rate datacube.
    template = np.zeros(ds.vx.shape)
    template = template[:,::upscale, ::upscale]
    template = template[:,1:-1,1:-1]
    template[template==0] = np.nan
    
    # Rearrange the coordinates so they fit how we calculate the strain rates
    xx = ds.x
    yy = ds.y

    Y = yy.values[::upscale]
    Y = Y[1:-1]
      
    X = xx.values[::upscale]
    X = X[1:-1]

    # Create empty datacube to host the generated strain rates
    strain_components = xr.Dataset(
        data_vars=dict(
            ezz=(["time", "y", "x"], template.copy()),
            divflux=(["time", "y", "x"], template.copy()),
            e1=(["time", "y", "x"], template.copy()),
            e2=(["time", "y", "x"], template.copy()),
            
        ),
        coords=dict(
            time=(["time"],ds.time.values),
            y=(["y"], Y),
            x=(["x"], X),
        ),
        attrs=dict(description=f"Strain Rate Datacube with an upscale = {upscale}"),
    )
    
    
    # Loop through the strain calculation
    for i in range(len(ds.time.values)-amount):

        # At every loop, increment the spatial step
        start = str(ds.time.values[i].astype(f'M8[{TYPE}]'))
        end = str(ds.time.values[i+amount].astype(f'M8[{TYPE}]'))

        # Calculate the strains
        ezz, divflux, e1, e2 = Strains_Calculator(ds, start, end, upscale, TYPE, Plotting, flux_div)

        # Save the calculated strains in the datacube
        strain_components.ezz[i] = ezz
        strain_components.divflux[i] = divflux
        strain_components.e1[i] = e1
        strain_components.e2[i] = e2
        print(start)
    
    # Export the datacube
    strain_components.to_netcdf(f'{pathsave}{list_subfolders[0]}/{amount}_{TYPE}/{sdate}_{edate}_{upscale}.nc')

    return strain_components


### Instructions

<br>

The following function can:
- Calculate strain rates
- Calculate flux divergence
- Plot the strain rates
- Plot arrows of the strain rates (beware, this option can take a lot of time depending on your upscale)

<br>

You can choose your input as it is written in the following cell. Just change statements to "True" or "False". It will print the date of each step processed.





In [ ]:
datacube = Runner(Plotting = True, upscale = 5, sdate = sdate, edate = edate, flux_div = flux_div)